In [1]:
!pip install arxiv

In [2]:
!pip install wikipedia

In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper = api_wrapper)

In [16]:
wiki.name

'wikipedia'

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)


vectordb = FAISS.from_documents(documents,OllamaEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x74df5a6287d0>)

In [6]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool = create_retriever_tool(retriever, "langsmith_search", 
                      "Search for information from LangSmith. For any question about LangSmith, you must use this tool")

In [7]:
retrieval_tool.name

'langsmith_search'

In [8]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
tools = [wiki, arxiv , retrieval_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/kaarthiksiva/PREFIX=/home/kaarthiksiva/anaconda3/envs/Django/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=1000, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information from LangSmith. For any question about LangSmith, you must use this tool', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x74df6f799a80>, retriever=VectorStoreRetriever(tags=['F

In [27]:
from dotenv import load_dotenv
load_dotenv()
import os

from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model = "llama3", temperature = 0)

In [30]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:
# ### Agents
# from langchain.agents import create_openai_tools_agent
# agent = create_openai_tools_agent(llm,tools,prompt)

# from langchain_community.agents import create_
# agent = create_pbi_agent(llm,tools,prompt)

from langgraph.prebuilt import create_react_agent
from langchain.agents import create_agent
llm_with_tools = llm.bind_tools(tools)
agent_executor = create_react_agent(llm, tools)

NotImplementedError: 

In [35]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executer = AgentExecutor(agent = agent, tools = tools, verbose= True)
agent_executer

NameError: name 'agent' is not defined

In [28]:
response = agent_executer.invoke({"input": "Tell me about LangSmith"})
response



> Entering new AgentExecutor chain...


ValueError: This output parser only works on ChatGeneration output

In [ ]:
### Agents
from langchain.agents import create_openai_tools_agent
from langchain_community.agent_toolkits import create_pbi_agent
agent = create_pbi_agent(llm,tools,prompt)